In [2]:
import tensorflow as tf
import numpy as np
import plotly.figure_factory as ff
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, InputLayer, Activation
from keras.optimizers import RMSprop, Adam
import h5py
import matplotlib.pyplot as plt
from keras import backend as K

Using TensorFlow backend.


In [51]:
#MNIST Dataset Load

# batch_size = 128
# num_classes = 10
# epochs = 10

# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# x_train = x_train.reshape(60000, 784)
# x_test = x_test.reshape(10000, 784)
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# # One hot conversion
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)



from sklearn import datasets
from sklearn.utils import resample
from keras.utils import to_categorical
import numpy as np
n_samples = 2000
#Play with different datasets.
#feat, labels = datasets.make_classification(n_features=2, n_classes=2, n_redundant=0, n_informative=1, random_state=1, n_clusters_per_class=1, n_samples=n_samples)
feat, labels = datasets.make_moons(n_samples=n_samples, noise=0.05)
#feat, labels = datasets.make_circles(n_samples=400, factor=.3, noise=.05)
#feat, labels = datasets.make_blobs(n_samples=n_samples)
new_feat, new_labels = resample(feat, labels, n_samples=10000, replace=True)
new_labels = to_categorical(new_labels, num_classes=2)

#labels = labels.reshape([-1,1])
#print(feat.shape, labels.shape)
#((x,y) for zip(feat,labels) if labels==1)
#feat_small = feat[np.where(labels==1)]
# lab_small = np.zeros(feat_small.shape[0])
# lab_small = to_categorical(lab_small, num_classes=1)
# print(feat_small.shape, lab_small.shape)
#feat_small, label = ((x,y) for x,y in zip(feat,labels) if y==1)

#feat, labels = datasets.make_blobs(n_samples=n_samples)
labels = to_categorical(labels, num_classes=2)
print(feat.shape, labels.shape)


(2000, 2) (2000, 2)


In [52]:
K.clear_session()
model = Sequential()

###Model###
model.add(Dense(2, input_dim=2, activation='relu'))
model.add(Dense(2, input_dim=2, activation='relu'))
model.add(keras.layers.add(model.layers[-1], model.layers[-1]))
#model.add(Dropout(0.5))
model.add(Dense(2, input_dim=2, activation='relu'))
model.add(Dense(2, input_dim=2, activation='relu'))
model.add(Dense(2, input_dim=2, activation='relu'))
#model.add(Activation('tanh'))
# model.add(Dense(2, input_dim=2))
# model.add(Activation('tanh'))
# model.add(Dense(2, input_dim=2))
# model.add(Activation('tanh'))
# model.add(Dense(2, input_dim=2))
# model.add(Activation('tanh'))
# model.add(Dense(2, input_dim=2))
# model.add(Activation('tanh'))
model.add(Dense(2))
model.add(Activation('softmax'))
model.summary()

#tfcall = keras.callbacks.TensorBoard(log_dir='../plots/keras600newlogs', histogram_freq=1, write_graph=True)

model.compile(loss='categorical_crossentropy',optimizer=Adam(), metrics=['accuracy'])

history = model.fit(new_feat, new_labels,
                    #batch_size=batch_size,
                    epochs=20, #EPOCHS
                    verbose=1)
                    #validation_data=(x_test, y_test),
                    #callbacks=[tfcall])
#score = model.evaluate(x_test, y_test, verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 6         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 6         
__________

KeyboardInterrupt: 

In [43]:
def get_layer(inp, out, layer_array):
    return K.function([layer_array.layers[inp].input], [layer_array.layers[out].output])

In [44]:
#first = get_layer(0, 0, model)
# second = get_layer(0,2, model)
# third = get_layer(0,4, model)
# out1 = first([x_test])
# out2 = second([x_test])
# out3 = third([x_test])
# print(out1[0].shape, out2[0].shape, out3[0].shape)

activations = []
for layer in range(len(model.layers)):
    extract_func = get_layer(0,layer, model)
    activations.append(extract_func([new_feat])[0])

In [45]:
activations[0].shape

(10000, 2)

In [23]:
import sys
sys.path.append('../scripts/')
from utils import *

In [46]:
morphs(activations, np.argmax(new_labels, axis=1), skip=1, last_layer=True)

This is the format of your plot grid:
[ (1,1) x1,y1 ]     [ (1,2) x2,y2 ]     [ (1,3) x3,y3 ]     [ (1,4) x4,y4 ]     [ (1,5) x5,y5 ]     [ (1,6) x6,y6 ]     [ (1,7) x7,y7 ]     [ (1,8) x8,y8 ]     [ (1,9) x9,y9 ]     [ (1,10) x10,y10 ]



In [47]:
mesh = [(a,b) for a in np.arange(-10, 10, 1) for b in np.arange(-10, 10, 1)]
mesh = np.asarray(mesh)
print(mesh.shape)
def morphs_nolab(activations, skip=2, last_layer=False):
    #If last dimension is something other than 2, change it. otherwise 0.
    trace_list = []
    n_rows = 1 if ((len(activations)) < 10) else -(-len(activations)//10)
    fig = tools.make_subplots(rows=n_rows, cols=(10))
    last_dim = (2 if last_layer==False else 0)
    for i in range(1, len(activations)):
        #print((-(-i//10)), i%10, i-1, (i//2)+1)
        #        fig.append_trace(go.Scatter(x=activations[i-1][:,0], y = activations[i-1][:,1] , mode='markers'), -(-i//10), (10 if i%10==0 else i%10))
        temp_fig = ff.create_quiver(activations[i-1][:,0], activations[i-1][:,1], activations[i][:,0], activations[i][:,1],line=dict(width=1))
        fig.append_trace(temp_fig['data'][0], -(-i//10), (10 if i%10==0 else i%10))
    fig['layout'].update(height=n_rows*500)
    fig['layout'].update(width=3500)
    plot(fig)

(400, 2)


In [48]:
meshVelocities = []
#meshVelocities.append(sess.run(x, feed_dict={x:mesh}))
for layer in range(len(model.layers)):
    extract_func = get_layer(0,layer, model)
    meshVelocities.append(extract_func([mesh])[0])
#print(len(meshVelocities))
morphs_nolab(meshVelocities, skip=1, last_layer=True)

This is the format of your plot grid:
[ (1,1) x1,y1 ]     [ (1,2) x2,y2 ]     [ (1,3) x3,y3 ]     [ (1,4) x4,y4 ]     [ (1,5) x5,y5 ]     [ (1,6) x6,y6 ]     [ (1,7) x7,y7 ]     [ (1,8) x8,y8 ]     [ (1,9) x9,y9 ]     [ (1,10) x10,y10 ]

